# LECTURE 04: Perceptron and Generalized Linear Model

### Boilerplate

In [162]:
import torch # type: ignore
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt # type: ignore
from typing import Callable

from sklearn.datasets import make_classification

### GML with hypothesis as input

In [163]:

def updateParameterMatrix(Y: np.ndarray,
                          X: np.ndarray,
                          theta: np.ndarray,
                          hypothesisMatrixFunction: Callable[[np.ndarray, np.ndarray], np.ndarray | float],
                          learningRate: float) -> np.ndarray:
    
    """ Compute theta prime using GLM update rule

    Update parameter matrix using GLM update rule with the given hypothesis function

    Args:
        Y: Output vector where each element is output of one sample
        X: Input matrix of shape (m_sample, n_features) where each row
           represents the input of one sample.
        theta: parameter vector containing weight of each feature
        hypothesis: function that maps parameter matrix θ and input matrix X to a
                    hypothesis output
    
    Returns:
        numpy.ndarray: Array of updated parameter vector theta(θ') of size (n_features,)
    """
    m = Y.shape[0]
    eta = theta.T @ X.T
    prediction = hypothesisMatrixFunction(eta)
    error = prediction - Y
    gradient = X.T @ error.T

    return theta - learningRate / m * gradient




### Softmax Regression

In [164]:
def hypothesisMatrixSoftmax(eta: np.ndarray) -> np.ndarray:

    """ Computes hypothesis matrix for softmax function with given eta (η = θ.T dot X.T)
    
    Args:
        eta: η = θ.T dot X.T, input for hypothesis function, shape (k_classes, m_samples)
    
    Returns:
        numpy.ndarray: Softmax hypothesis matrix, shape (k_classes, m_samples)
    """

    etaExp              = np.exp(eta)

    # Denominator: sum over classes (axis=1), shape (m,)
    denominator = np.sum(etaExp, axis=0, keepdims=True)  # shape: (1, m)

    # Element-wise division, broadcasted over each row
    hypothesis = etaExp / denominator  # shape: (m, k)
    
    return hypothesis

def softmaxRegression(X:np.ndarray, Y: np.ndarray, alpha: float, epochs: int) -> np.ndarray:
    n, k = X.shape[1], Y.shape[0]
    theta = np.zeros((n, k))
    
    for epoch in range(epochs):
        theta = updateParameterMatrix(Y, X, theta, hypothesisMatrixSoftmax, alpha)
    return theta

In [ ]:
# Function to generate synthetic dataset
def generate_synthetic_data(n_samples=100, n_features=5, n_classes=3, random_state=42):
    """Generate synthetic classification data with one-hot encoded labels"""
    X, y_integer = make_classification(
        n_samples=n_samples, 
        n_features=n_features, 
        n_classes=n_classes, 
        n_informative=3, 
        random_state=random_state
    )
    
    # Convert integer labels to one-hot encoding
    Y = np.zeros((n_samples, n_classes))
    for i in range(n_samples):
        Y[i, y_integer[i]] = 1
    
    return X, Y

# Test the softmax regression implementation
def test_softmax_regression():
    # Generate synthetic data
    X, Y = generate_synthetic_data(n_samples=200, n_features=5, n_classes=3)
    
    # Normalize features for better convergence
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    
    # Hyperparameters
    alpha = 0.1  # Learning rate
    epochs = 1000  # Number of iterations
    
    # Run softmax regression
    theta = softmaxRegression(X, Y, alpha, epochs)
    
    # Print the output (theta values)
    print(f"Trained theta values: \n{theta}")
    
    # Make predictions
    eta = X @ theta
    predictions = hypothesisMatrixSoftmax(eta)
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(Y, axis=1)
    
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == actual_classes)
    print(f"Accuracy: {accuracy:.4f}")
    
    # Visualize decision boundaries if features are 2D
    if X.shape[1] >= 2:
        plot_decision_boundary(X[:, :2], actual_classes, theta[:2, :], predictions)
    
    return theta, accuracy

def plot_decision_boundary(X, y, theta, predictions):
    """Plot the decision boundary for 2D data"""
    # Set figure size
    plt.figure(figsize=(10, 8))
    
    # Create a mesh grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.02  # Step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # Plot the decision boundary
    plt.subplot(1, 2, 1)
    plt.title('Decision Boundary')
    
    # Plot data points
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    
    # Plot the confidence of predictions
    plt.subplot(1, 2, 2)
    plt.title('Prediction Confidence')
    
    # Get the confidence of the predictions (max probability)
    confidence = np.max(predictions, axis=1)
    
    # Plot data points with color based on confidence
    scatter = plt.scatter(X[:, 0], X[:, 1], c=confidence, cmap='viridis', 
                         edgecolors='k', alpha=0.7)
    plt.colorbar(scatter, label='Confidence')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    
    plt.tight_layout()
    plt.show()

theta, accuracy = test_softmax_regression()
print(f"Final theta shape: {theta.shape}")
print(f"Final accuracy: {accuracy:.4f}")